In [1]:
import pandas as pd
from os import path
from sklearn.neighbors import KNeighborsClassifier

## Load and clean the data

In [3]:
filename = path.join(".", "resources", "exoplanet_data.csv")

df = pd.read_csv(filename)
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
# Use the seven most important features identified in the random forest model
target = df['koi_disposition']
data = df[['koi_fpflag_co', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_model_snr', 'koi_prad', 'koi_prad_err2', 'koi_duration_err2']]
data.head()

,koi_fpflag_co,koi_fpflag_nt,koi_fpflag_ss,koi_model_snr,koi_prad,koi_prad_err2,koi_duration_err2
0,0,0,0,25.8,2.83,-0.19,-0.11600
1,0,0,1,76.3,14.60,-1.31,-0.03410
2,0,0,1,505.6,33.46,-2.83,-0.00537
3,0,0,0,40.9,2.75,-0.35,-0.04200
4,0,0,0,40.2,2.77,-0.30,-0.06730


## Split and scale the data

In [5]:
# Split the data into train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, train_size=0.8, random_state=12)

In [6]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Find the best K

In [7]:
train_scores = []
test_scores = []
for k in range(1, 18, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

k: 1, Train/Test Score: 1.000/0.816
k: 3, Train/Test Score: 0.900/0.853
k: 5, Train/Test Score: 0.889/0.856
k: 7, Train/Test Score: 0.883/0.860
k: 9, Train/Test Score: 0.881/0.863
k: 11, Train/Test Score: 0.878/0.865
k: 13, Train/Test Score: 0.876/0.864
k: 15, Train/Test Score: 0.877/0.866
k: 17, Train/Test Score: 0.878/0.866


In [8]:
knn = KNeighborsClassifier(n_neighbors=17)
knn.fit(X_train_scaled, y_train)
print('k=17 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))

k=17 Test Acc: 0.866


## Tune the model with GridSearchCV

In [9]:
# https://medium.com/@erikgreenj/k-neighbors-classifier-with-gridsearchcv-basics-3c445ddeb657
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors': [3,9,31,51],
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan']}
# grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1)
grid = GridSearchCV(knn, param_grid, verbose=1)

In [10]:
gs = grid.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    1.9s finished


In [11]:
gs.best_score_

0.8007841463356147

In [12]:
gs.best_estimator_

KNeighborsClassifier(metric='manhattan', n_neighbors=9, weights='distance')

In [13]:
gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}

In [14]:
predictions = gs.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.74      0.66      0.70       339
     CONFIRMED       0.68      0.81      0.74       363
FALSE POSITIVE       0.93      0.88      0.91       697

      accuracy                           0.81      1399
     macro avg       0.78      0.78      0.78      1399
  weighted avg       0.82      0.81      0.81      1399

